# Imports

In [ ]:
import numpy as np
import pandas as pd
import os
import random
import shutil
import glob
import yaml
import torch
import matplotlib.pyplot as plt
import xml.etree.ElementTree as ET
import cv2

from PIL import Image
from matplotlib import pyplot as plt
from matplotlib import patches
from pathlib import Path
from sklearn.model_selection import train_test_split

# Install YOLOv8

In [ ]:
%pip install ultralytics

In [ ]:
import ultralytics #提供了多种计算机视觉工具和深度学习模型，如YOLOv5目标检测模型、图像分类模型等，可用于图像处理和深度学习任务
from ultralytics import YOLO
ultralytics.checks()

# Create Directory

In [ ]:
from pathlib import Path

directory = "archive"
image_directory = directory + "/images"
annotation_directory = directory + "/annotations"

"""
Path(annotation_directory) 创建了一个Path对象，表示annotation_directory变量所指定的目录。

.glob('**/*.xml') 是Path对象上的方法调用，它返回一个生成器，该生成器会产生所有匹配指定模式的文件
（在此处，即所有扩展名为.xml的文件）。

list() 将生成器转换为一个Path对象的列表，并将其存储在annotations变量中。
"""
annotations = list(Path(annotation_directory).glob('**/*.xml'))
print("存储文件路径，如", annotations[0])
print(len(annotations))

# Data Preprocessing

In [ ]:
"""
把xml文件中对图片的描述转成 dataframe 的形式
"""
class_id = {
    "with_mask" : 0,
    "mask_weared_incorrect" : 1,
    "without_mask" : 2
}

data_dict = {
    'filename': [],
    'label': [],
    'class_id': [],
    'width': [],
    'height': [],
    'bboxes': []
}
for annotation_path in annotations:
    tree = ET.parse(annotation_path) # 用于解析一个XML文件并返回一个ElementTree对象，该对象表示整个XML文档的树形结构
    root = tree.getroot() # 获取XML根节点
    filename = root.find('filename').text # 提取 root ElementTree对象中第一个出现的 "filename" 元素的文本内容
    for obj in root.findall("object"):
        label = obj.find("name").text
        
        bbox = []
        # bndbox has xmin, ymin, xmax, ymax
        bndbox_tree = obj.find('bndbox')
        bbox.append(int(bndbox_tree.find('xmin').text))
        bbox.append(int(bndbox_tree.find('ymin').text))
        bbox.append(int(bndbox_tree.find('xmax').text))
        bbox.append(int(bndbox_tree.find('ymax').text))
        size = root.find('size')
        
        data_dict['filename'].append(filename)
        data_dict['width'].append(int(size.find('width').text))
        data_dict['height'].append(int(size.find('height').text))
        data_dict['label'].append(label)
        data_dict['class_id'].append(class_id[label])
        data_dict['bboxes'].append(bbox)

df_data = pd.DataFrame(data_dict)

df_data.head()

In [ ]:
df_data.isna().sum() # calculate the number of missing values (NaN)

In [ ]:
df_data.info()

In [ ]:
df_data.label.unique() # three labels

In [ ]:
print(f"Total 'without_mask' labels: {sum(df_data.label == 'without_mask')}")
print(f"Total 'mask_weared_incorrect' labels: {sum(df_data.label == 'mask_weared_incorrect')}")
print(f"Total 'with_mask' labels: {sum(df_data.label == 'with_mask')}")

In [ ]:
#!pip install matplotlib Pillow

# Visualize Data

In [ ]:
def show_random_images_with_bbox(df):
    all_images = os.listdir(image_directory)
    random_image_filename = random.sample(all_images, 4) # 在所有照片中随机取4张
    fig, ax = plt.subplots(nrows=2, ncols=2, figsize=(10, 10)) # 创建一块画布，2*2个子图
    for i, filename in enumerate(random_image_filename): # 将一个可遍历的数据对象(如列表、元组或字符串)组合为一个索引序列，同时列出数据和数据下标
        selected_df = df[df['filename'] == filename]
        #print(selected_df)
        
        image = Image.open(image_directory + '/' + filename)
        
        ax.flat[i].imshow(image) # 显示图片
        ax.flat[i].axis(False) # 坐标轴
        
        image_bboxes = []
        for df_index in range(0, len(selected_df)):
            color = "g"
            if selected_df.iloc[df_index].class_id == 1: color = "y"
            elif selected_df.iloc[df_index].class_id == 2: color = "r"
            
            x_min, y_min, x_max, y_max = selected_df.iloc[df_index].bboxes
            
            rect = patches.Rectangle([x_min, y_min], x_max-x_min, y_max-y_min, 
                             linewidth=2, edgecolor=color, facecolor="none") # 画矩形
            ax.flat[i].add_patch(rect) # 添加在图片上
            
show_random_images_with_bbox(df_data)

In [ ]:
show_random_images_with_bbox(df_data)

In [ ]:
# we need to convert our bbox format to yolo as the current one that we have is on pascal_voc
def pascal_voc_to_yolo_bbox(bbox_array, w, h):
    x_min, y_min, x_max, y_max = bbox_array
    
    x_center = ((x_max + x_min) / 2) / w
    y_center = ((y_max + y_min) / 2) / h
    
    width = (x_max - x_min) / w
    height = (y_max - y_min) / h
    
    return [x_center, y_center, width, height]

# Split Train, Val, Test images

Create our directory to place our images

In [ ]:
train_path = "YOLOv8/working/datasets/train"
valid_path = "YOLOv8/working/datasets/valid"
test_path = "YOLOv8/working/datasets/test"

!os.mkdir("YOLOv8/working/datasets")
!os.mkdir(train_path)
!os.mkdir(valid_path)
!os.mkdir(test_path)

We need to move our respective images to its folder and create label file for our image to use in YOLOv8

In [ ]:
train, test = train_test_split(df_data.filename.unique(), test_size=0.2, random_state=23) # 划分训练集，测试集
train, valid = train_test_split(train, test_size=0.15, random_state=23) # 在训练集上划分训练和验证集

def copy_image_file(image_items, folder_name):
    """把相应的照片复制到对应的目录"""
    for image in image_items:
            image_path = image_directory + "/" + image
            new_image_path = os.path.join(folder_name, image)
            shutil.copy(image_path, new_image_path)

def create_label_file(image_items, folder_name):
    """为每张照片制作txt文件写入标签和位置信息"""
    for image in image_items:
        fileName = Path(image).stem # 文件名
        df = df_data[df_data['filename'] == image]
        with open(folder_name + "/" + fileName +'.txt', 'w') as f:
            for i in range(0, len(df)):
                bbox = pascal_voc_to_yolo_bbox(df.iloc[i]['bboxes'], df.iloc[i]['width'], df.iloc[i]['height'])
                bbox_text = " ".join(map(str, bbox))
                txt = str(df.iloc[i]['class_id'])+ " " + bbox_text
                f.write(txt)
                if i != len(df) - 1:
                    f.write("\n")
                

copy_image_file(train, train_path)
copy_image_file(valid, valid_path)
copy_image_file(test, test_path)

create_label_file(train, train_path)
create_label_file(valid, valid_path)
create_label_file(test, test_path)

In [ ]:
def walk_through_dir(filepath):
    """
    遍历指定目录并打印每个子目录中包含的子目录和以".png"为扩展名的图像文件的数量
    """
    for dirpath, dirnames, filenames in os.walk(filepath):
        print(f"There are {len(dirnames)} directories and {len(glob.glob(filepath + '/*.png', recursive = True))} images in '{dirpath}'.")
    
walk_through_dir(train_path)
walk_through_dir(valid_path)  
walk_through_dir(test_path)  

# Create YAML file

In [ ]:
classes = list(df_data.label.unique())
class_count = len(classes)
facemask_yaml = f"""
    train: train
    val: valid
    test: test
    nc: {class_count}
    names:
        0 : with_mask
        1 : mask_weared_incorrect
        2 : without_mask
    """

with open('YOLOv8/working/datasets/facemask.yaml', 'w') as f:
    f.write(facemask_yaml)

%cat 'YOLOv8/working/datasets/facemask.yaml'

# Train

In [ ]:
model = YOLO("YOLOv8/working/datasets/yolov8n.pt")
# facemask.yaml 是一个数据集的配置文件（configuration file），用于指定训练、验证和测试数据集的路径以及一些其他参数
model.train(data="YOLOv8/working/datasets/facemask.yaml", epochs=50) # 训练

In [ ]:
model.val(data="YOLOv8/working/datasets/facemask.yaml") # 验证

# Validation

In [ ]:
confusion_matrix = Image.open("YOLOv8/working/runs/detect/train/confusion_matrix.png")
plt.figure(figsize=(20,10))
plt.imshow(confusion_matrix)
plt.axis(False)
plt.show()

In [ ]:
val_label = Image.open("YOLOv8/working/runs/detect/train/val_batch0_labels.jpg")
val_pred = Image.open("YOLOv8/working/runs/detect/train/val_batch0_pred.jpg")

plt.figure(figsize=(20,10))
plt.imshow(val_label)
plt.title("Label")
plt.axis(False)
plt.show()

plt.figure(figsize=(20,10))
plt.imshow(val_pred)
plt.title("Prediction")
plt.axis(False)
plt.show()

# Predict

In [ ]:
# 这个权重文件通常是在之前的训练过程中得到的，包含了模型的各个参数和参数的初始值。
model = YOLO(model="YOLOv8/working/runs/detect/train/weights/best.pt") # 将预训练的权重文件加载到模型中

In [ ]:
val_label = Image.open("/content/drive/MyDrive/Colab Notebooks/564 DL/working/runs/detect/train/val_batch0_labels.jpg")
val_pred = Image.open("/content/drive/MyDrive/Colab Notebooks/564 DL/working/runs/detect/train/val_batch0_pred.jpg")

plt.figure(figsize=(20,10))
plt.imshow(val_label)
plt.title("Label")
plt.axis(False)
plt.show()

plt.figure(figsize=(20,10))
plt.imshow(val_pred)
plt.title("Prediction")
plt.axis(False)
plt.show()

In [ ]:
predicted_image = Image.open("YOLOv8/working/runs/detect/predict/image0.jpg")
plt.figure(figsize=(10,10))
plt.imshow(predicted_image)
plt.title("Prediction")
plt.axis(False)
plt.show()

predicted_image = Image.open("YOLOv8/working/runs/detect/predict/image1.jpg")
plt.figure(figsize=(10,10))
plt.imshow(predicted_image)
plt.title("Prediction")
plt.axis(False)
plt.show()

Cross Validation

In [ ]:
from sklearn.model_selection import KFold

# K 折交叉验证
kfold2 = KFold(n_splits=3, shuffle=True, random_state=23)

# 获取文件名列表
file_list = df_data.filename.unique()

for fold, (train_index, test_index) in enumerate(kfold2.split(file_list)):
    # 划分训练集和测试集
    train_files = file_list[train_index]
    test_files = file_list[test_index]
    # 在训练集上划分训练集和验证集
    train_index, valid_index = train_test_split(train_index, test_size=0.15, random_state=23)
    train_files = file_list[train_index]
    valid_files = file_list[valid_index]

    # 创建文件夹
    train_path = f"YOLOv8/kfold2/train{fold}"
    valid_path = f"YOLOv8/kfold2/valid{fold}"
    test_path = f"YOLOv8/kfold2/test{fold}"
    os.makedirs(train_path, exist_ok=True)
    os.makedirs(valid_path, exist_ok=True)
    os.makedirs(test_path, exist_ok=True)

    # 复制图像文件到对应文件夹
    copy_image_file(train_files, train_path)
    copy_image_file(valid_files, valid_path)
    copy_image_file(test_files, test_path)

    # 生成标签文件
    create_label_file(train_files, train_path)
    create_label_file(valid_files, valid_path)
    create_label_file(test_files, test_path)

    # 创建配置文件
    classes = list(df_data.label.unique())
    class_count = len(classes)
    facemask_yaml = f"""
        train: {train_path}
        val: {valid_path}
        test: {test_path}
        nc: {class_count}
        names:
            0 : with_mask
            1 : mask_weared_incorrect
            2 : without_mask
        """

    with open(f'YOLOv8/kfold2/facemask{fold}.yaml', 'w') as f:
        f.write(facemask_yaml)


    # 训练模型并记录日志
    model = YOLO("YOLOv8/working/datasets/yolov8n.pt")
    model.train(data=f"YOLOv8/kfold2/facemask{fold}.yaml", epochs=50)

In [ ]:
# 这个权重文件通常是在之前的训练过程中得到的，包含了模型的各个参数和参数的初始值。
model = YOLO(model="/content/drive/MyDrive/Colab Notebooks/564 DL/working/runs/detect/train/weights/best.pt") # 将预训练的权重文件加载到模型中

In [ ]:
filenames = glob.glob(test_path+"/*.png", recursive=False)
test_image1 = cv2.imread(filenames[0])
test_image2 = cv2.imread(filenames[1])

results = model.predict([test_image1, test_image2], save=True, line_thickness=1)

In [ ]:
predicted_image = Image.open("runs/detect/predict/image0.jpg")
plt.figure(figsize=(10,10))
plt.imshow(predicted_image)
plt.title("Prediction")
plt.axis(False)
plt.show()

predicted_image = Image.open("runs/detect/predict/image1.jpg")
plt.figure(figsize=(10,10))
plt.imshow(predicted_image)
plt.title("Prediction")
plt.axis(False)
plt.show()